In [0]:
%sql
use catalog dkushari_uc;

### Encrypter Function

In [0]:
%sql
CREATE OR REPLACE FUNCTION pepsi_demo.encrypt(query_text STRING) RETURNS STRING
COMMENT 'Registering Encrypt Function'
RETURN SELECT base64(aes_encrypt(query_text, secret('dkushari-scope', 'key'),'ECB')) AS ColumnName;


### Decrypter Function

In [0]:
%sql
CREATE OR REPLACE FUNCTION pepsi_demo.decrypt(enc_data STRING) RETURNS STRING
COMMENT 'Registering Decrypt Function'
RETURN SELECT aes_decrypt(unbase64(enc_data), secret('dkushari-scope', 'key'),'ECB') AS ColumnName;

### Returns Dynamic SQL statement based on predicate and Delta table version

In [0]:
%sql
CREATE OR REPLACE FUNCTION pepsi_demo.sql_query_fn(catalog_name string, schema_name string, table_name string, predicate string, version_number int)
RETURNS STRING
LANGUAGE PYTHON
AS
$$
if version_number is not None:
  table_ver = f"version as of {version_number}"
  sql_query_text = f"select * from {catalog_name}.{schema_name}.{table_name} {table_ver} where {predicate}"
  print(sql_query_text)
else:
  sql_query_text = f"select * from {catalog_name}.{schema_name}.{table_name} where {predicate}"

return sql_query_text
$$;


### Returns encrypted SQL Statement

In [0]:
%sql
CREATE OR REPLACE FUNCTION pepsi_demo.sql_query_generate(catalog_name string, schema_name string, table_name string, predicate string, version_number int)
RETURNS STRING
LANGUAGE SQL
RETURN SELECT pepsi_demo.encrypt(pepsi_demo.sql_query_fn(catalog_name, schema_name, table_name, predicate, version_number));

### Execute the dynamic SQL Statement

In [0]:
%sql
DECLARE OR REPLACE CATALOG_NAME STRING ;
DECLARE OR REPLACE SCHEMA_NAME STRING ;
DECLARE OR REPLACE TABLE_NAME STRING ;
DECLARE OR REPLACE VERSION_NUMBER INT DEFAULT 0;  
SET VAR VERSION_NUMBER=:VERSION_NUMBER;
SET VAR CATALOG_NAME=:CATALOG_NAME;
SET VAR SCHEMA_NAME=:SCHEMA_NAME;
SET VAR TABLE_NAME=:TABLE_NAME;
VALUES(VERSION_NUMBER);
DECLARE OR REPLACE PREDICATE STRING DEFAULT NULL ;
SET VAR PREDICATE=(select predicate from fgac.predicate_mapping where catalog_name = CATALOG_NAME and schema_name = SCHEMA_NAME and table_name=TABLE_NAME);
VALUES (PREDICATE);
DECLARE OR REPLACE DYNAMIC_GENERATE_SQL STRING DEFAULT "SELECT 1";
VALUES (DYNAMIC_GENERATE_SQL);
SET VAR DYNAMIC_GENERATE_SQL=(select pepsi_demo.sql_query_generate(CATALOG_NAME, SCHEMA_NAME, TABLE_NAME, PREDICATE, VERSION_NUMBER));
VALUES (DYNAMIC_GENERATE_SQL);
DECLARE OR REPLACE DECRYPT_SQL STRING DEFAULT "SELECT 1";
SET VAR DECRYPT_SQL = (SELECT pepsi_demo.decrypt(DYNAMIC_GENERATE_SQL));
values (DECRYPT_SQL);
EXECUTE IMMEDIATE DECRYPT_SQL;

record_id,first_name,last_name,date_of_birth,age,sex,address,ssn,region
4,Doe,Johnson,1985-01-01,38,M,123 Main St,333-33-3333,West
14,Doe,Brown,1990-01-01,33,F,456 Elm St,444-44-4444,West
35,Doe,Smith,1985-01-01,38,F,123 Main St,222-22-2222,West
38,John,Smith,1980-01-01,43,F,456 Elm St,333-33-3333,West
39,Doe,Smith,1980-01-01,43,M,123 Main St,111-11-1111,West
40,Alice,Johnson,1990-01-01,33,F,456 Elm St,333-33-3333,West
4,Doe,Johnson,1985-01-01,38,M,123 Main St,333-33-3333,West
14,Doe,Brown,1990-01-01,33,F,456 Elm St,444-44-4444,West
35,Doe,Smith,1985-01-01,38,F,123 Main St,222-22-2222,West
38,John,Smith,1980-01-01,43,F,456 Elm St,333-33-3333,West


### Use the dataframe downstream

In [0]:
_sqldf.createOrReplaceTempView("CUST_DATA")

In [0]:
spark.sql("select * from CUST_DATA").display()

record_id,first_name,last_name,date_of_birth,age,sex,address,ssn,region
4,Doe,Johnson,1985-01-01,38,M,123 Main St,333-33-3333,West
14,Doe,Brown,1990-01-01,33,F,456 Elm St,444-44-4444,West
35,Doe,Smith,1985-01-01,38,F,123 Main St,222-22-2222,West
38,John,Smith,1980-01-01,43,F,456 Elm St,333-33-3333,West
39,Doe,Smith,1980-01-01,43,M,123 Main St,111-11-1111,West
40,Alice,Johnson,1990-01-01,33,F,456 Elm St,333-33-3333,West
4,Doe,Johnson,1985-01-01,38,M,123 Main St,333-33-3333,West
14,Doe,Brown,1990-01-01,33,F,456 Elm St,444-44-4444,West
35,Doe,Smith,1985-01-01,38,F,123 Main St,222-22-2222,West
38,John,Smith,1980-01-01,43,F,456 Elm St,333-33-3333,West
